<a href="https://colab.research.google.com/github/Ferdib-Al-Islam/gpu_parallelization/blob/master/compute_co_occurrence_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-6ubwctra
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-6ubwctra
  Stored in directory: /tmp/pip-ephem-wheel-cache-9hctsh1w/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [0]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <ctype.h>



__device__ char* own_strcat(char *strg1, char *strg2)
{
    char *start = strg1;
 
    while(*strg1 != '\0')
    {
        strg1++;
    }
 
    while(*strg2 != '\0')
    {
        *strg1 = *strg2;
        strg1++;
        strg2++;
    }
 
    *strg1 = '\0';
    return start;
}


__device__ char* own_strcpy(char *destination, char *source)
{
    char *start = destination;
 
    while(*source != '\0')
    {
        *destination = *source;
        destination++;
        source++;
    }
 
    *destination = '\0'; 
    return start;
}

__device__ int own_strcmp(char *strg1, char *strg2)
{
    while( ( *strg1 != '\0' && *strg2 != '\0' ) && *strg1 == *strg2 )
    {
        strg1++;
        strg2++;
    }
 
    if(*strg1 == *strg2)
    {
        return 0; 
    }
 
    else
    {
        return *strg1 - *strg2;
    }
}

__global__ void co_matrix(char **k_words, int k_index, char **k_neighbours, int k_neighbours_index){
    int i, j, k;
    int occurance_count;
    char co_occur_matrix[200];
    
    i = blockIdx.x; 
    
    for (j=0; j<k_index; j++)
    {
        char w_i[200];
        own_strcpy(w_i, k_words[i]);

        own_strcat(w_i, " & ");

        char w_j[200];
        own_strcpy(w_j, k_words[j]);
        own_strcpy(co_occur_matrix, own_strcat(w_i, w_j));       

        occurance_count=0;

        for(k=0;k<k_neighbours_index;k++){
            if(own_strcmp(co_occur_matrix, k_neighbours[k]) == 0){
                occurance_count++;
            }
        }
        printf("%s --> %d, ", co_occur_matrix, occurance_count);
    } 
}

int main()
{
    int max_words = 1000;
    FILE *f;
    int i;
    int distinct;
    int index;
    int side_index;
    
    
    char **side;   
    side =  (char**) malloc (max_words*10*sizeof(char*)); 
    
    char word[200];
    side_index = 0;
    f = fopen("input.txt", "r");
    char store_prev_val[10][200];
    strcpy(store_prev_val[0], "");
    while (fscanf(f, "%s", word) != EOF)
    {            
        char w_i[200]="";
        side =  (char**) realloc (side, (side_index+1)*sizeof(char*));
        *(side+side_index) =  (char*) malloc (200*sizeof(char)); 
        strcat(w_i, store_prev_val[0]);
        strcat(w_i, " & ");
        strcat(w_i, word);
        strcpy(store_prev_val[0], word);
        strcpy(*(side+side_index), w_i);
        side_index++;
    }

    fclose(f);
    
    
    char **words;
    words =  (char**) malloc (max_words*sizeof(char*));    
  
   
    char punct[5];
          
    
    f = fopen("input.txt", "r");
    index = 0;
    
    while (fscanf(f, "%s", word) != EOF)
    {
        int len=strlen(word);
        if(ispunct(word[len-1])){
            word[len-1]='\0';
        }
        distinct = 1;
        for (i=0; i<index && distinct; i++)
        {
            if (strcmp(words[i], word) == 0)
                distinct = 0;
        }

        if (distinct) 
        {
            words =  (char**) realloc (words, (index+1)*sizeof(char*));
            *(words+index) =  (char*) malloc (200*sizeof(char)); 
            strcpy(*(words+index), word);
            index++;
        }
    }

    fclose(f);
    
   
    char **device_side;
       
   
    cudaMalloc((void**)&device_side, side_index*sizeof(char*));
    
    for(i=0;i<side_index;i++){
        char *temp;
        cudaMalloc((void**)&(temp), 200*sizeof(char));
        cudaMemcpy(temp, *(side+i), 200*sizeof(char), cudaMemcpyHostToDevice);
        cudaMemcpy(device_side+i, &temp, sizeof(char*), cudaMemcpyHostToDevice);
    }
    
    
    char **device_words;
    cudaMalloc((void**)&device_words, index*sizeof(char*));
   
    
    for(i=0;i<index;i++){
        char *temp;
        cudaMalloc((void**)&(temp), 200*sizeof(char));
        cudaMemcpy(temp, *(words+i), 200*sizeof(char), cudaMemcpyHostToDevice);
        cudaMemcpy(device_words+i, &temp, sizeof(char*), cudaMemcpyHostToDevice);
    }
    
    
    co_matrix <<<index,1>>>(device_words, index, device_side, side_index);
    
    
    cudaDeviceReset();
    return 0;
}

'I & I --> 0, deep & I --> 0, eat & I --> 0, rice & I --> 0, potato & I --> 0, like & I --> 0, learning & I --> 0, programming & I --> 0, deep & like --> 0, I & like --> 2, eat & like --> 0, potato & like --> 0, rice & like --> 0, like & like --> 0, learning & like --> 0, programming & like --> 0, deep & deep --> 0, I & deep --> 0, potato & deep --> 0, rice & deep --> 0, eat & deep --> 0, like & deep --> 1, learning & deep --> 0, programming & deep --> 0, deep & learning --> 0, I & learning --> 0, potato & learning --> 0, rice & learning --> 0, eat & learning --> 0, like & learning --> 0, learning & learning --> 0, programming & learning --> 0, deep & programming --> 0, I & programming --> 0, potato & programming --> 0, rice & programming --> 0, eat & programming --> 0, learning & programming --> 0, like & programming --> 0, programming & programming --> 0, deep & eat --> 0, I & eat --> 2, potato & eat --> 0, rice & eat --> 0, eat & eat --> 0, like & eat --> 0, learning & eat --> 0, pr